$$
\newcommand{\trace}[1]{\mathrm{tr}\left({#1}\right)}
\newcommand{\expval}[1]{\left\langle{#1}\right\rangle}
\newcommand{\comm}[2]{\left[{#1},{#2}\right]}
\newcommand{\acomm}[2]{\left\{{#1},{#2}\right\}}
\newcommand{\dissip}[1]{\mathcal{D}\left({#1}\right)\left[\rho\right]}
\newcommand{\bop}{\hat{b}}
\newcommand{\bopn}[1]{\bop^{#1}}
\newcommand{\bdag}{\bop^\dagger}
\newcommand{\bdagn}[1]{\bop^{\dagger {#1}}}
\newcommand{\odv}[2]{\frac{\mathrm{d}{#1}}{\mathrm{d}{#2}}}
$$

<div align="left">

<img src="logo.svg" style="width:20%">

</div>

### **Welcome abroad!**
`boson_ladder` is _fully_ based on `SymPy`. However, there is no need to specifically import anything from `SymPy` to use the features of this package, except if you want to use `sympy.Symbol` as variables.

This tutorial contains four topics:
-   The bosonic ladder operator objects
-   Normal ordering
-   Evaluating commutators
-   Expectation value evolution in the Lindblad master equation framework

We hope you learn something new and have fun!

\- The authors

In [1]:
import pybolano as bl
import sympy as sm

latex = sm.latex

---

### **The bosonic ladder operator objects**

In `SymPy`, the ladder operators are available as `sympy.physics.secondquant.AnnihilateBoson` and `sympy.physics.secondquant.CreateBoson`. In this package, the recommended way to get these objects is by calling [`boson_ladder.ops`](https://github.com/hendry24/boson_ladder/blob/main/boson_ladder/utils/operators.py?plain=1#L19):

In [2]:
b_1, bd_1 = bl.ops(1)
b_2, bd_2 = bl.ops(2)

b_1

AnnihilateBoson(1)

In [3]:
bd_1

CreateBoson(1)

Using ``SymPy`` we can also print the object as a LaTeX code:

There are several advantages of calling this function over calling the `SymPy` constructors: 
-   You do not need to manually import `sympy.physics.secondquant` which requires a separate import. 
-   You also only need to call one thing instead of two. 
-   As shown above, you can input any nonzero integer as a subscript without the constructor raising an error when printing the latex output (this may just be a bug):

In [4]:
from sympy.physics.secondquant import AnnihilateBoson
try:
    latex(AnnihilateBoson(1))
except:
    print("An exception is raised!")

An exception is raised!


If you are working with single-mode systems, you can remove the subscript by specifying no input:

In [5]:
b, bd = bl.ops()
b

AnnihilateBoson()

In [6]:
bd

CreateBoson()

which cannot be done in the current version of `SymPy`:

In [7]:
from sympy.physics.secondquant import AnnihilateBoson
try:
    AnnihilateBoson()
except:
    print("Whoa, there!")

Whoa, there!


Other than the construction process, the outputs of `ops` are really the `SymPy` objects:

In [8]:
isinstance(b, AnnihilateBoson)

True

---

### **The bread and butter**

The $j$-th mode boson annihilation $\hat{b}_j$ and creation $\hat{b}_j^\dagger$ operators (which we call the **ladder operators**, for brevity) are defined by their commutation relations
\begin{align}
    \left[\hat{b}_j,\hat{b}_k\right] =\left[\hat{b}_j^\dagger,\hat{b}_k^\dagger\right] &= 0
    \notag
    \\
    \left[\hat{b}_j,\hat{b}_k^\dagger\right] &= \delta_{jk}
    \notag
\end{align}
where $\delta_{jk}$ is the Kronecker delta. This package is based on the explicit formula by [Blasiak](https://arxiv.org/abs/quant-ph/0507206),
\begin{equation*}
    :\mathrel{\bdagn{r_M}\bop^{s_M}\dots \bdagn{r_2}\bop^{s_2}\bdagn{r_1}\bop^{s_1}}: = 
    \begin{cases}\displaystyle
        \bdagn{d_M}\sum_{k=s_1}^{s_1+s_2+\dots+s_M} S_{\bm{r},\bm{s}}(k)\bdagn{k}\bop^k,
        &
        d_M \geq 0
        \\
        \displaystyle
        \sum_{k=r_M}^{r_1+r_2+\dots+r_M} S_{\overline{\bm{s}},\overline{\bm{r}}}(k)\bdagn{k}\bop^k\ \bop^{-d_M},
        &
        d_M<0
    \end{cases}
\end{equation*}
where
\begin{equation*}
    d_l = \sum_{m=1}^l\left(r_m-s_m\right)
\end{equation*}
is the $l$-th excess (of creation operators) and 
\begin{equation*}
    S_{\bm{r},\bm{s}}(k) = \frac{1}{k!}\sum_{j=0}^k \binom{k}{j}\left(-1\right)^{k-j}\prod_{m=1}^M \left(d_{m-1}+j\right)_{s_m}
\end{equation*}
is called the generalized Stirling number.

---

### **Normal ordering**

Normal ordering means using the commutation relations to obtain an expression where all creation operators $b_j^\dagger$ are positioned to the left of all annihilation operators $b_j$. Other than making the expressions nicer to see, normal-ordered operators behave nicer in quantum mechanics (see the [optical equivalence theorem](https://en.wikipedia.org/wiki/Optical_equivalence_theorem)). 

The function [`normal_ordering`](https://github.com/hendry24/boson_ladder/blob/main/boson_ladder/core/do_commutator.py#L170) normal-orders the input expression. Assuming a polynomial in the ladder operators (the most general form), the algorithm separates each addend by its subscripts. For each factor of one subscript, Blasiak's formulae above are used to compute the normal-ordered form. The factors are then multiplied, and the products for different terms are summed to give an almost-normal-ordered expression. Lastly, the algorithm moves the operators with different indices (which commute) around to give a nice-looking output.

In [9]:
bl.normal_ordering(b_1 * bd_1 * b_1)

AnnihilateBoson(1) + CreateBoson(1)*AnnihilateBoson(1)**2

You can also use the shorthand `NO`,

In [10]:
bl.NO(b_1 * bd_1 * b_1)

AnnihilateBoson(1) + CreateBoson(1)*AnnihilateBoson(1)**2

**Multipartite input:**

In [11]:
bl.normal_ordering(b_2** 3 * b_1 * bd_2**2 * b_1**2)

6*AnnihilateBoson(1)**3*AnnihilateBoson(2) + 6*CreateBoson(2)*AnnihilateBoson(1)**3*AnnihilateBoson(2)**2 + CreateBoson(2)**2*AnnihilateBoson(1)**3*AnnihilateBoson(2)**3

NOTE: the subscripts are ordered based on occurence, as they are treated as `sympy.Symbol` even if a `Number` is input.

**Polynomial input:**

In [12]:
bl.normal_ordering(b_1**2 * bd_2**4 \
                    + 5 * b_2**2 * bd_1**5 * b_1 \
                    + b_2)

AnnihilateBoson(2) + 5*CreateBoson(1)**5*AnnihilateBoson(1)*AnnihilateBoson(2)**2 + CreateBoson(2)**4*AnnihilateBoson(1)**2

**Input with symbols:**

In [13]:
x = sm.Symbol("x")

bl.normal_ordering(x * b_1 * x**2 * bd_1**2)

2*x**3*CreateBoson(1) + x**3*CreateBoson(1)**2*AnnihilateBoson(1)

---

### **Normal ordering of commutators**

Commutators can be normal-ordered with the function [`NO_commutator`](https://github.com/hendry24/boson_ladder/blob/main/boson_ladder/core/do_commutator.py#L170) which simply calls `normal_ordering(A*B-B*A)` given the inputs `A` and `B`, most generally polynomials in ladder operators.

In [28]:
A = bd*b
B = b
bl.NO_commutator(A, B)

-AnnihilateBoson()

Let us try a more complex commutator:

In [18]:
A = bd**2 * b**3
B = b**5 * bd

bl.NO_commutator(A, B)

-60*AnnihilateBoson()**5 - 60*CreateBoson()*AnnihilateBoson()**6 - 7*CreateBoson()**2*AnnihilateBoson()**7

By default, the input is normal ordered. You can disable this by setting the parameter `normal_order = False`.

In [19]:
bl.NO_commutator(A, B)

-60*AnnihilateBoson()**5 - 60*CreateBoson()*AnnihilateBoson()**6 - 7*CreateBoson()**2*AnnihilateBoson()**7

**Multipartite input:**

In [20]:
A = bd_1*bd_2
B = b_1*b_2

bl.NO_commutator(A, B)

-1 - CreateBoson(1)*AnnihilateBoson(1) - CreateBoson(2)*AnnihilateBoson(2)

**Polynomial inputs:**

In [21]:
A = b_1 + 2*b_2**2
B = bd_1**2 * bd_1 + 2*bd_2*b_2

bl.NO_commutator(A, B)

8*AnnihilateBoson(2)**2 + 3*CreateBoson(1)**2

**Inputs with symbols:**

In [22]:
x = sm.Symbol("x")
A = x*b_1 
B = x**(0.5)*bd_1*b

bl.NO_commutator(A, B)

x**1.5*AnnihilateBoson()

---

### **Expectation value evolution in the Lindblad Master Equation Framework**

The master equation in the Lindblad form, or the Lindblad master equation (LME) is arguably the simplest extension of the Schr&ouml;dinger equation to open quantum systems (i.e. systems whose interaction with their environments are practically intractable). Instead of the wave function vector $\left|\psi\right\rangle$, we describe the system using density matrices $\rho$, which evolves according to
\begin{align}
    \frac{\mathrm{d}\rho}{\mathrm{d}t} = -\frac{i}{\hbar}\left[\hat{H},\rho\right] + \sum_j \gamma_j \mathcal{D}\left(\hat{O}_j, \hat{P}_j\right)[\rho]
\notag
\end{align}
where the term with the Hamiltonian $\hat{H}$ describes the closed system dynamics, and the rest describe the open system dynamics. Here, $\gamma_j$ are nonnegative scalars, while 
\begin{align}
    \mathcal{D}\left(\hat{O}_j, \hat{P}_j\right)\left[\rho\right] = \hat{O}_j\rho\hat{P}_j^\dagger - \frac{1}{2}\left\{\hat{P}_j^\dagger\hat{O}_j,\rho\right\} \notag
\end{align}
is the Liouvillian superator in the Lindblad form, or more concisely, the Lindblad dissipator. The operators $\hat{O}_j, \hat{P}_j$ describing the dissipator depends on the process (e.g. $\hat{O}=\hat{P}=\hat{b}$ for a one-quantum dissipation). Meanwhile, the scalars $\gamma_j$ can be interpreted as the rate of the processes.

It is usually interesting to compute the evolution of some expectation value. For a system described by the density matrix $\rho$, the expectation value of some quantity represented by the operator $\hat{A}$ is given by $\left\langle A \right\rangle=\mathrm{tr}\left(\rho\hat{A}\right)$. The evolution of $\left\langle A \right\rangle$ is given by

\begin{align}
\frac{\mathrm{d}\left\langle A\right\rangle}{\mathrm{d}t} &= \mathrm{tr}\left(\frac{\mathrm{d}\rho}{\mathrm{d}t}\hat{A}\right) \notag
\\
&= -\frac{i}{\hbar}\ \mathrm{tr}\left(\left[\hat{H},\rho\right]\hat{A}\right) + \sum_j\gamma_j\mathrm{tr}\left( \mathcal{D}\left(\hat{O}_j, \hat{P}_j\right)\left[\rho\right]\hat{A}\right) \notag
\end{align}

We call the first term on the RHS the "Hamiltonian trace" and the rest the "dissipator traces". It can be straightforwardly shown that

\begin{align}
\mathrm{tr}\left(\left[\hat{H},\rho\right]\hat{A}\right) &=\left\langle\left[\hat{A},\hat{H}\right]\right\rangle \notag
\\ \notag
\\
\mathrm{tr}\left({\mathcal{D}\left({\hat{O}_j},\hat{P}_j\right)\left[\rho\right]\hat{A}}\right) 
&= \frac{1}{2}\left\langle{\left[{\hat{P}_j^\dagger},{\hat{A}}\right]\hat{O}_j}\right\rangle+\frac{1}{2}\left\langle{\hat{P}_j^\dagger\left[{\hat{A}},{\hat{O}_j}\right]}\right\rangle
\notag
\end{align}
These identities make the basis for the evaluation of the Hamiltonian trace by [`Hamiltonian_trace`](https://github.com/hendry24/boson_ladder/blob/main/boson_ladder/core/Lindblad_ME.py#L21) and the dissipator traces by [`dissipator_trace`](https://github.com/hendry24/boson_ladder/blob/main/boson_ladder/core/Lindblad_ME.py#L64)&mdash;check them out! 

The two functions make up the main function in this section: [`LME_expval_evo`](https://github.com/hendry24/boson_ladder/blob/main/boson_ladder/core/Lindblad_ME.py#L123). You only need to input the Hamiltonian `H`, the list of dissipators and their process rates `D`, and the operator `A` corresponding to the quantity whose expectation value evolution is of your interest&mdash;the function will do the rest. To be more specific, `D` has the syntax `D = [[gamma_0, O_0, P_0], [gamma_1, O_1, P_1], [gamma_2, O_2, P_2], ...]`. You can enter an empty list if the system is closed, and omit `P_j` if `O_j=P_j`.

**Example 1: One-dimensional quantum simple harmonic oscillator**

Let us start simple with the simple harmonic oscillator. We have $\hat{H}=\hbar\omega_0\hat{b}^\dagger\hat{b}$ and no dissipators. The evolution of the expected phase point $\left\langle \hat{b} \right\rangle = \mathrm{tr}\left(\rho \hat{b}\right)$ is given by

In [23]:
hbar, omega_0 = sm.symbols(r"hbar omega_0")
b, bd = bl.ops()

H = hbar*omega_0*bd*b
D = []
A = b

bl.LME_expval_evo(H, D, A, hbar_is_one=False)

Eq(Derivative({\left\langle b_{} \right\rangle}, t), -I*omega_0*{\left\langle b_{} \right\rangle})

What about its energy, $\left\langle \hat{b}^\dagger \hat{b} \right\rangle$? We can just set $\hat{A}=\hat{b}^\dagger \hat{b}$. We have

In [24]:
A = bd*b

bl.LME_expval_evo(H, D, A)

Eq(Derivative({\left\langle {b^\dagger_{}} b_{} \right\rangle}, t), 0)

**Example 2: The quantum Rayleigh oscillator, quantized by Chia et al.**

Let us set $\hbar=1$, for simplicity. The Hamiltonian is given by
\begin{equation}
\begin{split}
    \hat{H} &= \omega_0\bdag\bop + i\frac{\mu}{12}\left(\bdag\bop^3-\bdagn{3}\bop\right)  \notag
    \\
    &\quad + i\frac{\mu}{24}\left(\bop^4-\bdagn{4}\right) - i\frac{\mu\left(q_0^2-1\right)}{4}\left(\bop^2-\bdagn{2}\right) \notag
\end{split}
\end{equation}

Meanwhile, the open system dynamics is specified by

\begin{align}
    \gamma_1 &= \mu\left(q_0^2-1\right),\quad \hat{O}_1 = \bdag  \notag
    \\
    \gamma_2 &= \frac{3\mu}{4}, \quad \hat{O}_2 = \bop^2 \notag
    \\
    \gamma_3 &= \mu, \quad \hat{O}_3 = \bdag\bop - \frac{\bdagn{2}}{2} \notag
\end{align}

The evolution of $\expval{\bop}$ is governed by

\begin{align}
    \odv{\expval{\bop}}{t} &= -i\omega_0\expval{\bop} + \frac{\mu}{2} \left(q_0^2-1\right) \left[\expval{\bop}+\expval{\bdag}\right] \notag
    \\
    &\quad -\frac{\mu}{6}\left[\expval{\bop^3}+\expval{\bdagn{3}}\right] - \frac{\mu}{2}\left[\expval{\bdag\bop^2}+\expval{\bdagn{2}\bop}\right] \notag
\end{align}

In [25]:
omega_0, mu, q_0 = sm.symbols(r"omega_0 mu q_0")
b, bd = bl.ops()

H = omega_0*bd*b \
    + sm.I*mu/12 * (bd*b**3 - bd**3*b) \
    + sm.I*mu/24 * (b**4-bd**4) \
    - sm.I*mu*(q_0**2-1)/4 * (b**2-bd**2)

D = [[mu*(q_0**2-1), bd],
     [3*mu/4, b**2],
     [mu, bd*b-bd**2/2]]

A = b

bl.LME_expval_evo(H, D, A)

Eq(Derivative({\left\langle b_{} \right\rangle}, t), mu*q_0**2*{\left\langle b_{} \right\rangle}/2 + mu*q_0**2*{\left\langle {b^\dagger_{}} \right\rangle}/2 - mu*{\left\langle b_{} \right\rangle}/2 - mu*{\left\langle b_{}^{3} \right\rangle}/6 - mu*{\left\langle {b^\dagger_{}} \right\rangle}/2 - mu*{\left\langle {b^\dagger_{}} b_{}^{2} \right\rangle}/2 - mu*{\left\langle {b^\dagger_{}}^{2} b_{} \right\rangle}/2 - mu*{\left\langle {b^\dagger_{}}^{3} \right\rangle}/6 - I*omega_0*{\left\langle b_{} \right\rangle})

**Example 3: A bipartite quantum battery with quadratic driving by Downing & Ukhtary**

A quantum battery is a quantum system which can store energy and whose energy can be extracted for useful work. This particular setup is specified by
\begin{equation}
\begin{split}
\notag
    \hat{H} &= \omega_c \bdag_c\bop_c + \omega_h\bdag_h\bop_h \\
    &\quad + g\left(\bdag_c\bop_h+\bdag_h\bop_c\right) + \frac{\Omega}{2}\delta(t)\left(\bdagn{2}_c+\bop^2_c\right)
\end{split}
\end{equation}
and
\begin{equation}
    \gamma_1 = \gamma, \quad \hat{O}_1 = \bop_c \notag
\end{equation}
where the quadratic pulse of strength $\Omega$ is applied at the beginning of the evolution. Here it is assumed to be a bang-bang pulse, as indicated by the Dirac delta function. The energy evolutions after the pulse application are given by
\begin{align}
\odv{\expval{\bdag_c\bop_c}}{t} &= -\gamma \expval{\bdag_c\bop_c} - ig\left[\expval{\bdag_c\bop_h}-\expval{\bdag_h\bop_c}\right]   \notag
\\
\odv{\expval{\bdag_h\bop_h}}{t} &= ig\left[\expval{\bdag_c\bop_h} - \expval{\bdag_h\bop_c}\right] \notag
\end{align} 

In [26]:
omega_c, omega_h, g, gamma = \
    sm.symbols(r"omega_c omega_h g gamma")
b_c, bd_c = bl.ops("c")
b_h, bd_h = bl.ops("h")

H = omega_c * bd_c*b_c \
    + omega_h * bd_h*b_h \
    + g*(bd_c*b_h + bd_h*b_c)

D = [[gamma, b_c]]

A = bd_c*b_c

bl.LME_expval_evo(H,D,A)

Eq(Derivative({\left\langle {b^\dagger_{\mathtt{\text{c}}}} b_{\mathtt{\text{c}}} \right\rangle}, t), -I*g*{\left\langle {b^\dagger_{\mathtt{\text{c}}}} b_{\mathtt{\text{h}}} \right\rangle} + I*g*{\left\langle {b^\dagger_{\mathtt{\text{h}}}} b_{\mathtt{\text{c}}} \right\rangle} - gamma*{\left\langle {b^\dagger_{\mathtt{\text{c}}}} b_{\mathtt{\text{c}}} \right\rangle})

In [27]:
A = bd_h*b_h

bl.LME_expval_evo(H,D,A)

Eq(Derivative({\left\langle {b^\dagger_{\mathtt{\text{h}}}} b_{\mathtt{\text{h}}} \right\rangle}, t), I*g*{\left\langle {b^\dagger_{\mathtt{\text{c}}}} b_{\mathtt{\text{h}}} \right\rangle} - I*g*{\left\langle {b^\dagger_{\mathtt{\text{h}}}} b_{\mathtt{\text{c}}} \right\rangle})

---

### **You made it here. Congratulations!**

You should now have a good idea of how to utilize the package. We would like to hear your thoughts on the package: bugs, bad call signatures, potential improvements, extra features, etc. We hope this package be of use to you.

Thank you very much.

---

**The authors:**
-   Hendry M. Lim
-   Donny Dwiputra
-   M. Shoufie Ukhtary
-   Ahmad R. T. Nugraha

_(Last updated: 23 December 2024)_